# Time Series Classificatino with PeMS Data from UCI Machine Learning Repository

## PeMS Data?
### 1. 출처 : UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/PEMS-SF)
- PeMS 데이터 출처 : California 교통국 웹사이트(http://pems.dot.ca.gov/)에서 데이터 다운로드 

### 2. 기간 : 2008년 1월 1일부터 3월 30일까지의 기간 (10분 마다 셈플링)
 
### 3. 센서의 수 : 963개
- 하루 당 record 수 : 144 (24 * 6) * 963
- 144 : 24시간 * 60분에서 (10분 마다 셈플링이기 때문에 6)

### 4. 데이터 : 0에서 1사이의 샌프란시스코 베이 지역 고속도로의 서로 다른 자동차 차선의 비율 

### 5. Label : 월요일 ~ 일요일 (1 ~ 7)


## Data Folder
### 데이터 : Train과 Test 에 대하여 Data와 Label 존재
### 총 Record 수 : 440 (Train : 263, Test : 173)
### 1. PEMS_train, PEMS_trainlabels : 각 행렬은 하루 동안 10 분마다 (144 열) 샘플링 된 센서들의 점유율 (963 줄 - 각 스테이션 / 감지기 당 하나), 측정의 각 날에 대해 데이터가 샘플 된 요일 (즉, 1 (월)과 7 (일) 사이의 정수를 설명)
### 2. PEMS_test, PEMS_testlabels : Train 데이터셋과 Label과 같지만 갯 수가 173개
### 3. randperm : 데이터를 섞는 데 사용된 순열 (날짜 순서대로 바꾸기 위해서는 Train data와 Test data를 합쳐서 다시 순서를 재정렬해야합니다.)
### 4. station_list :  각 센서의 ID들 리스트


## Data Processing

데이터가 매트랩기준으로 되어 있기 떄문에 처리를 해야합니다.

### 1. PEMS_train, PEMS_test
- []로 묶여져 있는 데이터가 263개와 173개가 있습니다. (Record 수 또는 총 일수를 의미합니다.)
- [] 안에는 ;로 구분되어 있는 행이 963개가 있습니다. (총 센서 갯 수 입니다.)
- ;로 구분되어 있는 행은 띄어쓰기로 144개가 구분되어 있습니다. (24시간 * 6개의 10분)

### 2. 나머지 데이터들
- []로 묶여져 있고 띄어쓰기로 구분되어 있습니다.

In [2]:
import json

def read_file(file_name):
    file = open(file_name, 'r')
    data = file.readlines()
    if "PEMS_train" in file_name or "PEMS_test" in file_name:
        temp_data = [i.replace('[', '').replace(']', '').split(';') for i in data]
        data = []
        for i in temp_data:
            elem = []
            for j in i:
                elem.append([float(k) for k in j.split(' ')])
            data.append(elem)
        data = pd.DataFrame(data=data, dtype='float32')
        return data
    data = data[0].replace('[', '').replace(']', '').split(' ')
    if data[0].isdigit():
        data = pd.DataFrame(data=data, dtype='int32')
    else:
        data = pd.DataFrame(data=data, dtype='float32')
    return data

In [3]:
import pandas as pd

rand_data = read_file('data/PEMS-SF/randperm')
station_data = read_file('data/PEMS-SF/stations_list')
train_data = read_file('data/PEMS-SF/PEMS_train')
train_label_data = read_file('data/PEMS-SF/PEMS_trainlabels')
test_data = read_file('data/PEMS-SF/PEMS_test')
test_label_data = read_file('data/PEMS-SF/PEMS_testlabels')

Normalization이 되어 있기 때문에 skip
왜 해야하는 지도 나중에 설명

train과 test도 나눠져 있음

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

In [4]:
train_data = train_data.transpose()
test_data = test_data.transpose()

In [4]:
temp_train_data = []
for i in range(train_data.shape[1]):
    elem = []
    print(i)
    for j in range(144):
        elem.append(pd.DataFrame(train_data[i].tolist())[j].tolist())
    temp_train_data.append(elem)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


KeyboardInterrupt: 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(64, input_shape=(144, train_data.shape[0])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)